# Logtalk tutorial

Logtalk is an *object-oriented logic programming language* that extends and leverages Prolog with modern code encapsulation and code reuse mechanisms without compromising its declarative programming features. Logtalk is implemented in highly portable code and can use most modern and standards compliant Prolog implementations as a backend compiler.

To keep its size reasonable, this tutorial necessarily assumes that the reader have a working knowledge of Prolog and is biased towards describing Logtalk object-oriented features.

Running this notebook assumes Logtalk is installed using one of the provided installers or by running the manual installation script. The default backend is SWI-Prolog but it can be changed in the fly by adding a code cell at the top and running one of the following queries: `eclipse`, `gnu`, `lvm`, `sicstus`, `trealla`, or `yap` (default is `swi`).


## Syntax

Logtalk uses standard Prolog syntax with the addition of a few operators and directives for a smooth learning curve and wide portability. One important consequence is that Prolog code can be easily encapsulated in objects with little or no changes. Moreover, Logtalk can transparently compile most Prolog modules as Logtalk objects.

The main operators are:

* `::/2` - sending a message to an object
* `::/1` - sending a message to _self_ (i.e. to the object that received the message being processed)
* `^^/1` - _super_ call (of an inherited or imported predicate)

Some of the most important entity and predicate directives will be introduced in the next sections.

## Entities and roles

Logtalk provides _objects_, _protocols_, and _categories_ as first-class entities. Relations between entities define _patterns of code reuse_ and the _roles_ played by the entities. For example, when an object _instantiates_ another object, the first object plays the role of an instance and the second object plays the role of a class. An _extends_ relation between two objects implies that both objects play the role of prototypes, with one of them extending the other, its parent prototype.

## Defining an object

An object encapsulates predicate _declarations_ and _definitions_. Objects can be created dynamically but are usually static and defined in source files. A single source file can contain any number of entity definitions. A simple object, defining a list member public predicate:

In [1]:
%%file lists.lgt

:- object(lists).

	:- public(member/2).
	member(Head, [Head| _]).
	member(Head, [_| Tail]) :-
		member(Head, Tail).

:- end_object.

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/lists.lgt loaded ]
% (0 warnings)

true

## Compiling and loading source files

In the notebook cell above, we used the `%%file FILE` *cell magic* to save the cell contents to a `lists.lgt` file and compile/load it when the cell is run. At a Logtalk top-level, assuming that the code above for the `list` object is saved in a `list.lgt` file, it can be compiled and loaded using the `logtalk_load/1` built-in predicate or its abbreviation, `{}/1`, with the file path as argument (the extension can be omitted):

```text
?- {list}.
yes
```

In general, entities may have dependencies on entities defined in other source files (e.g. library entities). To load a file and all its dependencies, the advised solution is to define a _loader_ file that loads all the necessary files for an application. A loader file is simply a source file, typically named `loader.lgt`, that makes calls to the `logtalk_load/1-2` built-in predicates, usually from an `initialization/1` directive for portability and standards compliance. Loader files are provided for all libraries, tools, and examples.

## Sending a message to an object

The `::/2` infix operator is used to send a message to an object. As in Prolog, we can backtrack for alternative solutions:

In [2]:
list::member(X, [1,2,3]).

X = 1

In [3]:
retry.

X = 2

In [4]:
retry.

X = 3

Encapsulation is enforced. A predicate can be declared _public_, _protected_, or _private_. It can also be _local_ when there is no scope directive for it. For example:

In [5]:
%%file scopes.lgt

:- object(scopes).

	:- private(bar/0).
	bar.

	local.

:- end_object.

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/scopes.lgt loaded ]
% (0 warnings)

true

Sending a message for a declared predicate that is out of scope results in a permission error:

In [6]:
catch(scopes::bar, Error, true).

Error = error(permission_error(access,private_predicate,bar/0),logtalk(scopes::bar,c(user,user,r(user,scopes,[],[]))))

Sending a message for a defined but not non-declared predicate results in a existence error:

In [7]:
catch(scopes::local, Error, true).

Error = error(existence_error(predicate_declaration,local/0),logtalk(scopes::local,c(user,user,r(user,scopes,[],[]))))

When the predicate in a message is unknown for the object (the role it plays determines the lookup procedures), we also get an error. For example:

In [8]:
catch(scopes::unknown, Error, true).

Error = error(existence_error(predicate_declaration,unknown/0),logtalk(scopes::unknown,c(user,user,r(user,scopes,[],[]))))

A subtle point is that predicate scope directives specify predicate _calling_ semantics, not _definition_ semantics. For example, if an object playing the role of a class declares a predicate private, the predicate can be defined in subclasses and instances *but* can only be called in its instances _from_ the class.

## Defining and implementing a protocol

Protocols contain predicate declarations that can be implemented by any number of objects and categories:

In [9]:
%%file lists.lgt

:- protocol(lists_protocol).

	:- public(member/2).

:- end_protocol.

:- object(lists,
	implements(lists_protocol)).

	member(Head, [Head| _]).
	member(Head, [_| Tail]) :-
		member(Head, Tail).

:- end_object.

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/lists.lgt compiled ]
% Redefining object lists
% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/lists.lgt reloaded ]
% (0 warnings)

true

The scope of the protocol predicates can be restricted using protected or private implementation. For example:

In [10]:
%%file stack.lgt

:- object(stack,
	implements(private::lists_protocol)).

:- end_object.

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/stack.lgt loaded ]
% (0 warnings)

true

In fact, all entity relations (in an entity opening directive) can be qualified as public (the default), protected, or private.

## Prototypes

An object without an _instantiation_ or _specialization_ relation with another object plays the role of a prototype.

In [11]:
%%file clyde.lgt

% clyde, our prototypical elephant
:- object(clyde).

	:- public(color/1).
	color(grey).

	:- public(number_of_legs/1).
	number_of_legs(4).

:- end_object.

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/clyde.lgt loaded ]
% (0 warnings)

true

 A prototype can also _extend_ another object, its parent prototype.

In [12]:
%%file fred.lgt

% fred, another elephant, is like clyde, except that he's white
:- object(fred,
	extends(clyde)).

	color(white).

:- end_object.

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/fred.lgt loaded ]
% (0 warnings)

true

When answering a message sent to an object playing the role of a prototype, we validate the message and look for an answer first in the prototype itself and, if not found, we delegate to the prototype parents if any:

In [13]:
fred::number_of_legs(N).

N = 4

In [14]:
fred::color(C).

C = white

A message is valid if the corresponding predicate is declared (and the sender is within scope) but it will fail, rather then throwing an error, if the predicate is not defined. This is called the _closed-world assumption_. For example, consider the following object, saved in a `foo.lgt` file:

In [15]:
%%file foo.lgt

:- object(foo).

	:- public(bar/0).

:- end_object.

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/foo.lgt loaded ]
% (0 warnings)

true

Loading the file and trying to call the `bar/0` predicate fails as expected. Note that this is different from calling an _unknown_ predicate, which results in an error:

In [16]:
foo::bar.

false

In [17]:
catch(foo::baz, Error, true).

Error = error(existence_error(predicate_declaration,baz/0),logtalk(foo::baz,c(user,user,r(user,foo,[],[]))))

## Classes and instances

In order to define objects playing the role of classes and/or instances, an object must have at least an instantiation or a specialization relation with another object. Objects playing the role of meta-classes can be used when we need to see a class also as an instance. We use the following example to also illustrate how to dynamically create new objects at runtime:

In [18]:
%%file classes.lgt

% a simple, generic, metaclass defining a new/2 predicate for its instances
:- object(metaclass,
	instantiates(metaclass)).

	:- public(new/2).
	new(Instance, Clauses) :-
		self(Class),
		create_object(Instance, [instantiates(Class)], [], Clauses).

:- end_object.

% a simple class defining age/1 and name/1 predicate for its instances
:- object(person,
	instantiates(metaclass)).

	:- public([
		age/1, name/1
	]).

	% a default value for age/1
	age(42).

:- end_object.

% a static instance of the class person
:- object(john,
	instantiates(person)).

	name(john).
	age(12).

:- end_object.

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/classes.lgt loaded ]
% (0 warnings)

true

When answering a message sent to an object playing the role of an instance, we validate the message by starting in its class and going up to its class superclasses if necessary. Assuming that the message is valid, then we look for an answer starting in the instance itself:

In [19]:
person::new(Instance, [name(paulo)]).

Instance = o1

In [20]:
o1::name(Name).

Name = paulo

In [21]:
o1::age(Age).

Age = 42

In [22]:
john::age(Age).

Age = 12

## Categories

A category is a fine grained unit of code reuse, used to encapsulate a _cohesive_ set of predicate declarations and definitions, implementing a _single_ functionality, that can be imported into any object. A category can thus be seen as the dual concept of a protocol. In the following example, we define categories representing car engines and then import them into car objects:

In [23]:
%%file categories.lgt

% a protocol describing engine characteristics
:- protocol(carenginep).

	:- public([
		reference/1,
		capacity/1,
		cylinders/1,
		horsepower_rpm/2,
		bore_stroke/2,
		fuel/1
	]).

:- end_protocol.

% a typical engine defined as a category
:- category(classic,
	implements(carenginep)).

	reference('M180.940').
	capacity(2195).
	cylinders(6).
	horsepower_rpm(94, 4800).
	bore_stroke(80, 72.8).
	fuel(gasoline).

:- end_category.

% a souped up version of the previous engine
:- category(sport,
	extends(classic)).

	reference('M180.941').
	horsepower_rpm(HP, RPM) :-
		^^horsepower_rpm(ClassicHP, ClassicRPM),	% "super" call
		HP is truncate(ClassicHP*1.23),
		RPM is truncate(ClassicRPM*0.762).

:- end_category.

% with engines (and other components), we may start "assembling" some cars
:- object(sedan,
	imports(classic)).

:- end_object.

:- object(coupe,
	imports(sport)).

:- end_object.

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/categories.lgt loaded ]
% (0 warnings)

true

Categories are independently compiled and thus allow importing objects to be updated by simple updating the imported categories without requiring object recompilation. Categories also provide _runtime transparency_. I.e. the category protocol adds to the protocol of the objects importing the category:

In [24]:
%%table
sedan::current_predicate(Predicate).

Predicate | 
:- | 
bore_stroke/2 | 
capacity/1 | 
cylinders/1 | 
fuel/1 | 
horsepower_rpm/2 | 
reference/1 | 

true

## Hot patching

Categories can be also be used for hot-patching objects. A category can add new predicates to an object and/or replace object predicate definitions. For example, consider the following object:

In [25]:
%%file buggy.lgt

:- set_logtalk_flag(complements, allow).

:- object(buggy).

	:- public(p/0).
	p :- write(foo).

:- end_object.

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/buggy.lgt loaded ]
% (0 warnings)

true

Assume that the object prints the wrong string when sent the message `p/0`:

In [26]:
buggy::p.

foo

true

If the object source code is not available and we need to fix an application running the object code, we can simply define a category that fixes the buggy predicate:

In [27]:
%%file patch.lgt

:- category(patch,
	complements(buggy)).

	% fixed p/0 def
	p :- write(bar).

:- end_category.

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/patch.lgt loaded ]
% (0 warnings)

true

We will now get:

In [28]:
buggy::p.

bar

true

As hot-patching forcefully breaks encapsulation, the `complements` compiler flag can be set (globally, per source file, or per-object) to allow, restrict, or prevent it.

## Parametric objects and categories

Objects and categories can be parameterized by using as identifier a compound term instead of an atom. Object and category parameters are _logical variables_ shared with all encapsulated predicates. An example with geometric circles:

In [29]:
%%file circles.lgt

:- object(circle(_Radius, _Color)).

	:- public([
		area/1, perimeter/1
	]).

	area(Area) :-
		parameter(1, Radius),
		Area is pi*Radius*Radius.

	perimeter(Perimeter) :-
		parameter(1, Radius),
		Perimeter is 2*pi*Radius.

:- end_object.

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/circles.lgt loaded ]
% (0 warnings)

true

Parametric objects are used just as any other object, usually providing values for the parameters when sending a message:

In [30]:
circle(1.23, blue)::area(Area).

Area = 4.752915525615998

Parametric objects also provide a simple way of associating a set of predicates with a plain Prolog predicate. Prolog facts can be interpreted as _parametric object proxies_ when they have the same functor and arity as the identifiers of parametric objects. Handy syntax is provided to for working with proxies. For example, assuming the following clauses for a `circle/2` predicate:

In [31]:
%%user

circle(1.23, blue).
circle(3.71, yellow).
circle(0.39, green).
circle(5.74, black).
circle(8.32, cyan).

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/user.lgt loaded ]
% (0 warnings)

true

With these clauses loaded, we can easily compute for example a list with the areas of all the circles:

In [32]:
findall(Area, {circle(_, _)}::area(Area), Areas).

Areas = [4.752915525615998,43.241195443275274,0.47783624261100754,103.50793811341508,217.4685833038541]

The `{Goal}::Message` construct proves `Goal`, possibly instantiating any variables in it, and sends `Message` to the resulting term.

## Events and monitors

Logtalk supports _event-driven programming_ by allowing defining events and monitors for those events. An event is simply the sending of a message to an object. Interpreting message sending as an atomic activity, a _before_ event and an _after_ event are recognized. Event monitors define event handler predicates, `before/3` and `after/3`, and can query, register, and delete a system-wide event registry that associates events with monitors. For example, a simple tracer for any message being sent using the `::/2` control construct can be defined as:

In [33]:
%%file tracer.lgt

:- object(tracer,
	% built-in protocol for event handlers
	implements(monitoring)).

	:- initialization(define_events(_, list, _, _, tracer)).

	before(Object, Message, Sender) :-
		write('call: '), writeq(Object), write(' <-- '), writeq(Message),
		write(' from '), writeq(Sender), nl.

	after(Object, Message, Sender) :-
		write('exit: '), writeq(Object), write(' <-- '), writeq(Message),
		write(' from '), writeq(Sender), nl.

:- end_object.

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/tracer.lgt loaded ]
% (0 warnings)

true

Assuming that the `tracer` object and the `list` object defined earlier are compiled and loaded, we can observe the event handlers in action by sending a message:

In [34]:
set_logtalk_flag(events, allow).

true

In [35]:
%%table
list::member(X, [1,2,3]).

X | 
:- | 
1 | 
2 | 
3 | 

call: list <-- member(_15340,[1,2,3]) from user
exit: list <-- member(1,[1,2,3]) from user
exit: list <-- member(2,[1,2,3]) from user
exit: list <-- member(3,[1,2,3]) from user

true

Events can be set and deleted dynamically at runtime by calling the `define_events/5` and `abolish_events/5` built-in predicates.

Event-driven programming can be seen as a form of _computational reflection_. But note that events are only generated when using the `::/2` message-sending control construct.

In [36]:
set_logtalk_flag(events, deny).

true

## Lambda expressions

Logtalk supports lambda expressions. Lambda parameters are represented using a list with the `(>>)/2` infix operator connecting them to the lambda. Some simple examples using library `meta`:

In [37]:
{meta(loader)}.

% [ /Users/pmoura/logtalk/library/meta/loader.lgt already loaded; skipping ]
% (0 warnings)

true

In [38]:
meta::map([X,Y]>>(Y is 2*X), [1,2,3], Ys).

Ys = [2,4,6]

Currying is also supported:

In [39]:
meta::map([X]>>([Y]>>(Y is 2*X)), [1,2,3], Ys).

Ys = [2,4,6]

Lambda free variables can be expressed using the extended syntax `{Free1, ...}/[Parameter1, ...]>>Lambda`.

## Macros

Terms and goals in source files can be _expanded_ at compile time by specifying a _hook object_ that defines term-expansion and goal-expansion rules.

Assume the following hook object, saved in a `my_macros.lgt` file, that expands clauses and calls to the `foo/1` local predicate:

In [40]:
%%file my_macros.lgt

:- object(my_macros,
	% built-in protocol for expanding predicates
	implements(expanding)).

	term_expansion(foo(Char), baz(Code)) :-
		% standard built-in predicate
		char_code(Char, Code).

	goal_expansion(foo(X), baz(X)).

:- end_object.

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/my_macros.lgt loaded ]
% (0 warnings)

true

Consider the following simple object, saved in a `source.lgt` file:

In [41]:
%%file source.lgt

:- set_logtalk_flag(hook, my_macros).

:- object(source).

	:- public(bar/1).
	bar(X) :- foo(X).

	foo(a). foo(b). foo(c).

:- end_object.

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/source.lgt loaded ]
% (0 warnings)

true

In [42]:
%%table
source::bar(X).

X | 
:- | 
97 | 
98 | 
99 | 

true

The Logtalk library provides support for combining hook objects using different workflows (for example, defining a pipeline of expansions).

## Further information

Visit the [Logtalk website](http://logtalk.org) for more information.